# Boolean Query

In this notebook, we will implement a Boolean information retrieval model by Elasticsearch.  
We have these steps: <br>

**1- Connect to the Elasticsearch Cluster and Create an Index <br>
2- Indexing Documents <br>
3- Boolean Retrieval (Fill where ever it says #TODO in this part)** <br>

Let me know if you have any problems with this notebook or implementation. <br>
**Telegram**: [@Mohammad_Ardestani4](https://t.me/Mohammad_Ardestani4) <br>
**Email**: mjavad.ardestani00@gmial.com

In [1]:
!pip install elasticsearch

In [2]:
from elasticsearch import Elasticsearch, helpers
import os, json, time
from datetime import datetime
from tqdm import tqdm 

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Ignore The Security Warnings
Here we ignore the security warnings. It's suggested that you don't run below cell until you ensure that all warnings are about security. 


In [3]:
import warnings
warnings.filterwarnings('ignore')

## Load Processed Data

In [4]:
import json
dataset_path = 'IR_data.json'
with open(dataset_path) as f:
    data = json.load(f)
print(data['0'].keys())

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])


## Connect to the Elasticsearch Cluster and Create an Index
After starting your Elasticsearch on your pc (localhost:9200 is the default), we have to connect to it via the following piece of code.


In [8]:
index_name = 'ph3'
es = Elasticsearch("http://localhost:9200", http_auth=('elastic', 'changeme'))
es.indices.create(index = index_name)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ph3'})

### Check the Cluster Status

In [9]:
dict(es.info())

{'name': '7f455992fd1b',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'L57jD4AiT8mGxqe04UISYg',
 'version': {'number': '8.2.2',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '9876968ef3c745186b94fdabd4483e01499224ef',
  'build_date': '2022-05-25T15:47:06.259735307Z',
  'build_snapshot': False,
  'lucene_version': '9.1.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

## Indexing Documents 

we can add our documents to created index in two ways:<br>
1- one by one in for loop. <br>
2- use [Bulk API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html#:~:text=Bulk%20APIedit,can%20greatly%20increase%20indexing%20speed.), which performs multiple indexing or delete operations in a single API call. This reduces overhead and can significantly increase indexing speed.

###  for loop by es.index 
**You dont have to run this cell**. 

In [ ]:
for i in tqdm(range(len(data))):
    es.index(index = index_name, id=i, document=data[str(i)])
    i = i + 1

100%|██████████| 12202/12202 [00:43<00:00, 279.57it/s]


###  Bulk API

In [10]:
%%time
from elasticsearch.helpers import bulk


def bulk_sync():
    actions = [
        {
            '_index': index_name,
            '_id':doc_id,
            '_source': doc
        } for doc_id,doc in data.items()
    ]
    bulk(es, actions)


CPU times: user 26 µs, sys: 4 µs, total: 30 µs
Wall time: 32.2 µs


In [11]:
start = time.time()
bulk_sync()
end = time.time()
print("Indexing all documents took about {:.2f} seconds".format(end - start))

Indexing all documents took about 6.26 seconds


### check index

In [12]:
es.count(index = index_name)

ObjectApiResponse({'count': 12202, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Boolean Retrieval 

### Bool Query Structure

It is built using one or more boolean clauses, each clause with a typed occurrence. The occurrence types are:

**must**: The clause (query) must appear in matching documents and will contribute to the score.
<br><br>
**filter**: The clause (query) must appear in matching documents. However unlike must the score of the query will be ignored. Filter clauses are executed in filter context, meaning that scoring is ignored and clauses are considered for caching.
<br><br>
**should**: The clause (query) should appear in the matching document.
<br><br>
**must_not**: The clause (query) must not appear in the matching documents. Clauses are executed in filter context meaning that scoring is ignored and clauses are considered for caching. Because scoring is ignored, a score of 0 for all documents is returned.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html). 

###  <span style="color:red"> TODO </span>
You should read about [match query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-top-level-params) and [match phrase query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html) then complete the below cell.

In [13]:
query= {
        "bool": {
          "should": [
              { 
                "match": {
                    "content": "عراق"
             }
              }, 
              
              { 
                    "match_phrase": {
          "content": " هواداران باشگاه"
      }
              },
   
          ],
          "must_not": [
              {
                  "match": {
                    "content": "ایران"
                }
              }
          ],
        },
    }

### Search query

The _source field contains the original JSON document body that was passed at index time. The _source field itself is not indexed (and thus is not searchable), but it is stored so that it can be returned when executing fetch requests, like get or search.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html#search-api-response-body).

In [14]:
res = es.search(index = index_name, query=query, _source= ["url"])
res = dict(res)

### Results

In [15]:
print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took']/1000))
for doc in res['hits']['hits']:
    print(doc['_source']['url'])

146 results in 0.023 s: 
https://www.farsnews.ir/news/14001027000416/فوتبال-عراق-حامی-یار-شهید-سردار-سلیمانی-عکس
https://www.farsnews.ir/news/14001007000723/طرح-قانونمند-شدن-کانون-هواداران-باشگاه‌ها-و-ایجاد-امکانات-رفاهی-برای
https://www.farsnews.ir/news/14001222000020/همکاری-خطیبی-و-آلومینیوم-بازهم-به-پایان-فصل-نرسید
https://www.farsnews.ir/news/14001217000662/پایان-سال-انتظار-عراق-AFC-میزبانی-همگروه-ایران-را-تایید-کرد
https://www.farsnews.ir/news/14001022000430/کادرفنی-عراق-پیش-از-دیدار-با-ایران-به-مشکل-خورد-عکس
https://www.farsnews.ir/news/14001104000059/دعوای-بزرگ-رقبای-ایران-در-فوتبال-بر-سر-رژیم-صهیونیستی
https://www.farsnews.ir/news/14000929000679/حاشیه-بازی-پرسپولیس-و-ویستا-توربین|-اعتراض-سرخ-پوشان-به-روشن-نشدن
https://www.farsnews.ir/news/14001107000613/اعتراف-رسانه-عراقی-فهد-طالب-با-سرسختی-دروازه-ما-را-نجات-داد-عکس-و-فیلم
https://www.farsnews.ir/news/14001217000221/دیدار-رقبای-ایران-با-تماشاگر-شد-امیدواری-عراقی‌ها-برای-صعود-به-جام
https://www.farsnews.ir/news/14001026000568/آم